In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [60]:
import json
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

In [35]:
file = open('/content/drive/MyDrive/AI Challenge/public_data/train/train_data.json')
data = json.load(file)
print(len(data))

46032


In [36]:
type(data)

list

In [37]:
data[0]

{'id': 14782,
 'Problem': 'a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?',
 'options': 'a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86',
 'Rationale': '"12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60 + 12 x x = 34 / 0.4 = 85 answer : d"',
 'correct': 'd',
 'diagramRef': '',
 'category': 'general'}

In [38]:
data[0]['Problem']

'a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?'

In [39]:
text = []
for i in range(len(data)):
  temp = "Problem: " + data[i]['Problem'] + '\n' + "Options: " + data[i]['options'] + '\n' + "Rationale: " + data[i]['Rationale'] + '\n' + "Correct: " + data[i]['correct'] + '\n' + "Diagram: " + data[i]['diagramRef'] + '\n' + "Category: " + data[i]['category']
  text.append(temp)

In [40]:
text[0]

'Problem: a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?\nOptions: a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86\nRationale: "12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60 + 12 x x = 34 / 0.4 = 85 answer : d"\nCorrect: d\nDiagram: \nCategory: general'

In [44]:
for i in range(len(data)):
  data[i]['text'] = text[i]

In [45]:
SPLIT = 0.8
train_data = data[:int(SPLIT*len(data))]
print(len(train_data))

36825


In [46]:
val_data = data[int(SPLIT*len(data)):]
print(len(val_data))

9207


In [47]:
with open('/content/drive/MyDrive/AI Challenge/train.json', 'w') as json_file:
  json.dump(train_data, json_file)

with open('/content/drive/MyDrive/AI Challenge/val.json', 'w') as json_file:
  json.dump(val_data, json_file)

In [50]:
data_train = load_dataset('json', data_files = '/content/drive/MyDrive/AI Challenge/train.json', split='train')
data_train

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['diagramRef', 'id', 'options', 'category', 'Rationale', 'Problem', 'text', 'correct'],
    num_rows: 36825
})

In [51]:
data_val = load_dataset('json', data_files = '/content/drive/MyDrive/AI Challenge/val.json', split='train')
data_val

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['diagramRef', 'id', 'options', 'category', 'Rationale', 'Problem', 'text', 'correct'],
    num_rows: 9207
})

In [53]:
tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardMath-7B-V1.0")
#tokenizer.pad_token = tokenizer.eos_token
tokenizer

(…)-V1.0/resolve/main/tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)Math-7B-V1.0/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)h-7B-V1.0/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)1.0/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='WizardLM/WizardMath-7B-V1.0', vocab_size=32000, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [54]:
context_length = 512 # 256
def tokenize(batch):
  outputs = tokenizer(
      batch['text'],
      truncation=True,
      max_length=context_length,
      return_overflowing_tokens=True,
      return_length=True
      #padding = 'max_length'
  )
  input_batch = []
  for length, input_ids, in zip(outputs['length'], outputs['input_ids']):
    input_batch.append(input_ids)
  result = {}
  result['input_ids'] = input_batch
  return result

In [57]:
tokenized_train_datasets = data_train.map(
    tokenize, batched=True, remove_columns=['diagramRef', 'id', 'options', 'category', 'Rationale', 'Problem', 'text', 'correct']
)
tokenized_train_datasets

Map:   0%|          | 0/36825 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 36992
})

In [58]:
tokenized_val_datasets = data_val.map(
    tokenize, batched=True, remove_columns=['diagramRef', 'id', 'options', 'category', 'Rationale', 'Problem', 'text', 'correct']
)
tokenized_val_datasets

Map:   0%|          | 0/9207 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 10802
})

In [59]:
tokenizer.decode(tokenized_train_datasets[0]['input_ids'])

'<s> Problem: a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?\nOptions: a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86\nRationale: "12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60 + 12 x x = 34 / 0.4 = 85 answer : d"\nCorrect: d\nDiagram: \nCategory: general'

In [61]:
train_data_path = '/content/drive/MyDrive/AI Challenge/tokenized_train'
tokenized_train_datasets.save_to_disk(train_data_path)

Saving the dataset (0/1 shards):   0%|          | 0/36992 [00:00<?, ? examples/s]

In [62]:
val_data_path = '/content/drive/MyDrive/AI Challenge/tokenized_val'
tokenized_val_datasets.save_to_disk(val_data_path)

Saving the dataset (0/1 shards):   0%|          | 0/10802 [00:00<?, ? examples/s]